# Coffee Machine Power Roll Up

Link to the example in the Cookbook:\
[https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_[...]5/views/_19_0_2_f46036d_1582648674041_192673_2975](https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_2_17_7_33_25_AM_3ccfaf88_159fe0d7ba9__7d4e_cae_tw_jpl_nasa_gov_127_0_0_1/master/documents/_18_5_2_e64033a_1513278979739_556269_28755/views/_19_0_2_f46036d_1582648674041_192673_2975)

## Structure
![img/coffee_structure.svg](img/coffee_structure.svg)
## Power Analysis
![img/coffee_analysis.svg](img/coffee_analysis.svg)
## Power Analysis Context
![img/coffee_constraint.svg](img/coffee_constraint.svg)

In [8]:
package 'CoffeeMaker Dynamic Power Roll-Up'{
    import ScalarValues::Real;
    import ScalarFunctions::sum;
    
    import 'Power Roll-Up Pattern'::*;
    import Behavior::*;
    import Structure::*;
    
    value type Power specializes Real;
    
    package 'Power Roll-Up Pattern' {
        
        block 'PowerRollUpPattern'{
            part subPower : 'PowerRollUpPattern'[*];
            
            value 'totalPower' : Power = sum(subPower::totalPower) + drawPower::power_draw;
            
            perform action drawPower : DrawPower;           
        }

        activity DrawPower{
            value power_draw : Power;
        }

    }
    
    package Behavior {
        activity 'Make Coffee' specializes DrawPower{
            value redefines power_draw = 0;
            
            first start;
            
            then action grindBeans : DrawPower {
                value redefines power_draw = 5;
            }

            then action heatWater : DrawPower {
                value redefines power_draw = 10;
            }

            then action brewCoffee : DrawPower {
                value redefines power_draw = 2;
                
                first start;
                then action filterCoffee : DrawPower {
                    value redefines power_draw = 1;
                }
                then done;
            }

            then action keepWarm : DrawPower {
                value redefines power_draw = 1;
            }
            
            then done;
        }
    }
    
    package Structure {
        block 'Coffee Machine' specializes PowerRollUpPattern{
            perform action 'make coffee' : 'Make Coffee' redefines drawPower;
            
            part 'coffee Grinder' subsets subPower {
                perform 'make coffee'::grindBeans redefines drawPower;
            }
            part 'coffee Pot' subsets subPower {
                perform 'make coffee'::heatWater redefines drawPower;
            }
            part brewer subsets subPower {
                perform 'make coffee'::brewCoffee redefines drawPower;
                
                part filter subsets subPower {
                    perform brewCoffee::filterCoffee redefines drawPower;
                }            
            }
            part 'water Heater' subsets subPower {
                perform 'make coffee'::keepWarm redefines drawPower;
            }            
        }
    }
    
    package 'Power Analysis' {     
        block 'Power Analysis Context'{
            value SIMULATION_TIME : Real;
            
            value totalPower : Power = 'coffee Machine'::totalPower;
            value power_per_time : Power = 'coffee Machine'::totalPower/SIMULATION_TIME;
            
            part 'coffee Machine' : 'Coffee Machine';           
        }
    }
}

Package CoffeeMaker Dynamic Power Roll-Up (faa6282e-6e0e-4d03-a01f-c01397bf4a93)
